#### Import modules

In [1]:
import os
import pandas as pd
import pickle

Read state codes mapping file:

In [2]:
name_to_code = []

with open(os.getcwd() + '/Data/name_to_code.pickle', 'rb') as pckl:
    name_to_code = pickle.load(pckl)
    
pckl.close()

name_to_code['Massachusetts']

'MA'

### States

#### Read data file

In [3]:
states_cases = pd.read_csv(os.getcwd() + '/Data/us-states.csv')
states_cases.shape

(33494, 5)

In [4]:
states_cases.tail()

,date,state,fips,cases,deaths
33489,2021-10-31,Virginia,51,924771,13907
33490,2021-10-31,Washington,53,727820,8686
33491,2021-10-31,West Virginia,54,272532,4426
33492,2021-10-31,Wisconsin,55,882618,9416
33493,2021-10-31,Wyoming,56,102926,1174


In [5]:
states_cases['state'].unique().shape

(56,)

#### Preprocess data

Remove AK, HI, and territories, and label DC:

In [6]:
states_cases['state_code'] = states_cases['state'].apply(lambda x: name_to_code[x] if x in name_to_code else 'EXCLUDE')
states_cases.loc[states_cases['state'] == 'District of Columbia', 'state_code'] = 'DC'
states_cases.loc[(states_cases['state_code'] == 'HI') | (states_cases['state_code'] == 'AK'), 'state_code'] = 'EXCLUDE'

states_cases.tail()

,date,state,fips,cases,deaths,state_code
33489,2021-10-31,Virginia,51,924771,13907,VA
33490,2021-10-31,Washington,53,727820,8686,WA
33491,2021-10-31,West Virginia,54,272532,4426,WV
33492,2021-10-31,Wisconsin,55,882618,9416,WI
33493,2021-10-31,Wyoming,56,102926,1174,WY


In [7]:
states_cases[states_cases['state_code'] == 'EXCLUDE']['state'].unique()

array(['Hawaii', 'Alaska', 'Puerto Rico', 'Virgin Islands', 'Guam',
       'Northern Mariana Islands', 'American Samoa'], dtype=object)

In [8]:
states_cases = states_cases[states_cases['state_code'] != 'EXCLUDE']
states_cases['state'].unique().shape

(49,)

Convert date from string to date type:

In [9]:
states_cases['date'] = pd.to_datetime(states_cases['date'])

Filter data to start from 2020-02-01, calculate new cases and deaths, and save:

In [10]:
states_cases = states_cases[states_cases['date'] >= '2020-02-01'].sort_values(['state_code', 'date'])

states_cases['new_cases'] = states_cases.groupby(['state_code'])['cases'].transform(lambda x: x.diff())
states_cases['new_deaths'] = states_cases.groupby(['state_code'])['deaths'].transform(lambda x: x.diff())

states_cases.tail()

,date,state,fips,cases,deaths,state_code,new_cases,new_deaths
33269,2021-10-27,Wyoming,56,101912,1174,WY,488.0,0.0
33325,2021-10-28,Wyoming,56,102403,1174,WY,491.0,0.0
33381,2021-10-29,Wyoming,56,102926,1174,WY,523.0,0.0
33437,2021-10-30,Wyoming,56,102926,1174,WY,0.0,0.0
33493,2021-10-31,Wyoming,56,102926,1174,WY,0.0,0.0


In [11]:
states_cases.drop(columns = ['state']).to_csv(os.getcwd() + '/Data/us_states_clean.csv')

### Counties

#### Read data file

In [3]:
counties_cases = pd.read_csv(os.getcwd() + '/Data/us-counties.csv')
counties_cases.shape

(1881596, 6)

In [4]:
counties_cases.tail()

,date,county,state,fips,cases,deaths
1881591,2021-11-03,Sweetwater,Wyoming,56037.0,7650,91.0
1881592,2021-11-03,Teton,Wyoming,56039.0,5234,14.0
1881593,2021-11-03,Uinta,Wyoming,56041.0,3871,27.0
1881594,2021-11-03,Washakie,Wyoming,56043.0,1763,35.0
1881595,2021-11-03,Weston,Wyoming,56045.0,1131,9.0


In [5]:
counties_cases['state'].unique().shape

(56,)

#### Preprocess data

* Remove AK, HI, and territories
* Label DC
* Rename county to append 'County' and state code

In [6]:
counties_cases['state_code'] = counties_cases['state'].apply(lambda x: name_to_code[x] if x in name_to_code else 'EXCLUDE')
counties_cases.loc[counties_cases['state'] == 'District of Columbia', 'state_code'] = 'DC'
counties_cases.loc[(counties_cases['state_code'] == 'HI') | (counties_cases['state_code'] == 'AK'), 'state_code'] = 'EXCLUDE'
counties_cases = counties_cases[counties_cases['state_code'] != 'EXCLUDE']
# counties_cases.loc[:, 'county'] = counties_cases['county'] + ' County, ' + counties_cases['state_code']

counties_cases['state'].unique().shape

(49,)

In [10]:
counties_cases = counties_cases[counties_cases['county'] != 'Unknown']
counties_cases.loc[:, 'county'] = counties_cases['county'] + ' County, ' + counties_cases['state_code']

counties_cases.tail()

,date,county,state,fips,cases,deaths,state_code
1881591,2021-11-03,"Sweetwater County, WY",Wyoming,56037.0,7650,91.0,WY
1881592,2021-11-03,"Teton County, WY",Wyoming,56039.0,5234,14.0,WY
1881593,2021-11-03,"Uinta County, WY",Wyoming,56041.0,3871,27.0,WY
1881594,2021-11-03,"Washakie County, WY",Wyoming,56043.0,1763,35.0,WY
1881595,2021-11-03,"Weston County, WY",Wyoming,56045.0,1131,9.0,WY


Convert date from string to date type:

In [11]:
counties_cases['date'] = pd.to_datetime(counties_cases['date'])

Filter data for the range 2020-02-01 to 2021-10-31, calculate new cases and deaths, and save:

In [12]:
counties_cases = counties_cases[(counties_cases['date'] >= '2020-02-01') & (counties_cases['date'] <= '2021-10-31')].sort_values(['state_code', 'date'])

counties_cases['new_cases'] = counties_cases.groupby(['county'])['cases'].transform(lambda x: x.diff())
counties_cases['new_deaths'] = counties_cases.groupby(['county'])['deaths'].transform(lambda x: x.diff())

counties_cases.tail()

,date,county,state,fips,cases,deaths,state_code,new_cases,new_deaths
1871844,2021-10-31,"Sweetwater County, WY",Wyoming,56037.0,7575,80.0,WY,0.0,0.0
1871845,2021-10-31,"Teton County, WY",Wyoming,56039.0,5204,14.0,WY,0.0,0.0
1871846,2021-10-31,"Uinta County, WY",Wyoming,56041.0,3829,26.0,WY,0.0,0.0
1871847,2021-10-31,"Washakie County, WY",Wyoming,56043.0,1730,35.0,WY,0.0,0.0
1871848,2021-10-31,"Weston County, WY",Wyoming,56045.0,1109,9.0,WY,0.0,0.0


In [13]:
counties_cases.drop(columns = ['state', 'state_code']).to_csv(os.getcwd() + '/Data/us_counties_clean.csv')